In [190]:
%dis 3088

Memory disassembled:
ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
           x3089: x302D  ST R0, TEMPX                              [line: 51]
           x308A: x302D  ST R0, TEMPY                              [line: 52]
ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
           x308C: x322C  ST R1, TEMPZ                              [line: 58]
           x308D: x1236  ADD R1, R0, #-10                          [line: 61]
           x308E: x0604  BRzp ASCII_LOOP (or 4)                    [line: 62]
           x308F: x2223  LD R1, ASCII_POS                          [line: 65]
           x3090: x1001  ADD R0, R0, R1                            [line: 66]
           x3091: xF021  OUT                                       [line: 67]
           x3092: xF025  HALT                                      [line: 69]
ASCII_LOOP: x3093: x2025  LD R0, TEMPZ                              [line: 74]
           x3094: x2221  LD R1, TEN  

In [600]:
.ORIG x3000

    BRnzp ASCII_CONVERT
    
;; ------------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;; ------------------------------------------------------------------

;; Subroutine to convert binary value into ASCII decimal and print.

ASCII_CONVERT:
    LD R0, CONVERT_VAL
    ST R0, TEMPX
    ST R0, TEMPY
    
ASCII_BASE: ;; Base case: if val < #10, add x30 and print

    ;; reset TEMPZ to #-1
    LD R1, NEG_ONE
    ST R1, TEMPZ
    
    ;; check val < #10
    ADD R1, R0, #-10
    BRzp ASCII_LOOP ;; if val >= #10, skip to main loop
    
    ;; convert R1 to ASCII and print
    LD R1, ASCII_POS
    ADD R0, R0, R1
    OUT R0
    
    HALT
    
ASCII_LOOP_REP: ;; start here if repeating non-base case

    ST R2, TEMPY ;; stores TEMPY from last loop

ASCII_LOOP: ;; Non-base case
    
    ;; TEMPZ = z*10
    LD R0, TEMPZ ;; R0 <- z
    LD R1, TEN ;; R1 <- 10
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1   = z*10
    ST R2, TEMPZ
    
    ;; R2 = y/10
    LD R0, TEMPY ;; R0 <- y
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1       = y/10
    ;;ST R2, TEMPY
    ;;ADD R2, R2, #0
    
    BRp ASCII_LOOP_REP ;; if quotient is positive, repeat loop
    
    ;; else if quotient is zero, use previous quotient (current val of TEMPY) as y
    
    ;; z = z/10 since the loop has to go one extra time to hit zero
    LD R0, TEMPZ ;; R0 <- z
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1    = z/10
    LD R0, QUOTIENT
    
    ;; R2 = y*z
    LD R1, TEMPY ;; R1 <- y
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1     = y*z
    
    LD R3, ASCII_POS
    ADD R0, R1, R3 ;; convert y to ASCII
    OUT R1
    
    LD R0, TEMPX
    ADD R0, R0, R2 ;; x-z: current state of val - (10^n * y)
    ST R0, TEMPX
    ST R0, TEMPY
    
    BRnzp ASCII_BASE ;; return to ASCII_BASE to check for base case
    
    
ASCII_POS:   .FILL x30
CONVERT_VAL:   .FILL #132 ;; value to be converted
NEG_ONE:   .FILL #-1
TEN:   .FILL #10
TEMPX:   .FILL #0 ;; stores present state of value to be converted, after subtraction
TEMPY:   .FILL #0 ;; stores present state of value to be converted, after division
TEMPZ:   .FILL #-1 ;; stores (negative) exponent of 10

;;-----------------------------------------------------------------
;; Signposts to PUSH and POP

PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0

.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [406]:
%bp x3088

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]


In [407]:
%bp x308B

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]


In [408]:
%bp x3093

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]


In [409]:
%bp x3094

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4) ASCII_LOOP: x3094: x2024  LD R0, TEMPZ                              [line: 78]


In [546]:
%bp x3099

Breakpoints
    1)            x3099: x243D  LD R2, MULT_PRODUCT                       [line: 83]


In [547]:
%bp x309F

Breakpoints
    1)            x3099: x243D  LD R2, MULT_PRODUCT                       [line: 83]
    2)            x309F: x245A  LD R2, QUOTIENT                           [line: 91]


In [411]:
%bp x30A0

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4) ASCII_LOOP: x3094: x2024  LD R0, TEMPZ                              [line: 78]
    5)            x3099: x243C  LD R2, MULT_PRODUCT                       [line: 83]
    6)            x30A0: x03F2  BRp ASCII_LOOP_REP                        [line: 95]


In [548]:
%bp x30A5

Breakpoints
    1)            x3099: x243D  LD R2, MULT_PRODUCT                       [line: 83]
    2)            x309F: x245A  LD R2, QUOTIENT                           [line: 91]
    3)            x30A5: x2454  LD R2, QUOTIENT                           [line: 104]


In [549]:
%bp x30AB

Breakpoints
    1)            x3099: x243D  LD R2, MULT_PRODUCT                       [line: 83]
    2)            x309F: x245A  LD R2, QUOTIENT                           [line: 91]
    3)            x30A5: x2454  LD R2, QUOTIENT                           [line: 104]
    4)            x30AB: x242B  LD R2, MULT_PRODUCT                       [line: 112]


In [414]:
%bp x30AE

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4) ASCII_LOOP: x3094: x2024  LD R0, TEMPZ                              [line: 78]
    5)            x3099: x243C  LD R2, MULT_PRODUCT                       [line: 83]
    6)            x30A0: x03F2  BRp ASCII_LOOP_REP                        [line: 95]
    7)            x30A5: x2453  LD R2, QUOTIENT                           [line: 104]
    8)            x30AA: x242B  LD R2, MULT_PRODUCT                       [line: 111]
    9)            x30AE: x2008  LD R0, TEMPX                              [line: 117]


In [415]:
%bp x30AF

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4) ASCII_LOOP: x3094: x2024  LD R0, TEMPZ                              [line: 78]
    5)            x3099: x243C  LD R2, MULT_PRODUCT                       [line: 83]
    6)            x30A0: x03F2  BRp ASCII_LOOP_REP                        [line: 95]
    7)            x30A5: x2453  LD R2, QUOTIENT                           [line: 104]
    8)            x30AA: x242B  LD R2, MULT_PRODUCT                       [line: 111]
    9)            x30AE: x2008  LD R0, TEMPX                              [line: 117]
    10)            x30AF: x1002  ADD R0, R0, R2                            [line: 118]


In [601]:
%exe

132============================================================
Computation completed
Instructions: 2715
Cycles: 18581 (0.009291 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x0032 R1: x0030 R2: xFFE2 R3: x0030 
R4: x0000 R5: x3002 R6: x30BD R7: x3093 


In [423]:
%cont

...breakpoint hit at x3099
Computation SUSPENDED
Instructions: 237
Cycles: 1711 (0.000856 milliseconds)

Registers:
PC: x3099
N: 1 Z: 0 P: 0 
R0: xFFF6 R1: x000A R2: x3066 R3: x0000 
R4: x0000 R5: x3002 R6: x30BC R7: x3099 


In [231]:
%dis 30B7 30B9

Memory disassembled:
TEMPY:     x30B7: x000C  NOOP - (no BR to x30C4) (or 12)           [line: 126]
TEMPZ:     x30B8: xFC18  ;; Invalid TRAP vector: x0018             [line: 127]
PUSH_IND1: x30B9: x3066  ST R0, x3120                              [line: 132]


In [14]:
%dis 30EC

Memory disassembled:
           x30EC: x14A1  ADD R2, R2, #1                            [line: 195]
           x30ED: x340A  ST R2, QUOTIENT                           [line: 196]
           x30EE: x0FF9  BRnzp DIVLOOP                             [line: 197]
DIVDONE:   x30EF: x16E0  ADD R3, R3, #0                            [line: 198]
           x30F0: x0604  BRzp DIVRET (or 4)                        [line: 199]
           x30F1: x2406  LD R2, QUOTIENT                           [line: 200]
           x30F2: x94BF  NOT R2, R2                                [line: 201]
           x30F3: x14A1  ADD R2, R2, #1                            [line: 202]
           x30F4: x3403  ST R2, QUOTIENT                           [line: 203]
DIVRET:    x30F5: x3003  ST R0, REMAINDER                          [line: 204]
           x30F6: x21C3  LD R0, POP_IND1                           [line: 205]
           x30F7: xC000  JMP R0                                    [line: 206]
QUOTIENT:  x30F8: x0000  NOOP -

In [371]:
%bp 3088

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]


In [372]:
%bp 308B

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]


In [373]:
%bp 3093

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]


In [374]:
%bp 30D2

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4)            x30D2: x14A1  ADD R2, R2, #1                            [line: 164]


In [375]:
%bp 30F5

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4)            x30D2: x14A1  ADD R2, R2, #1                            [line: 164]
    5)            x30F5: x3403  ST R2, QUOTIENT                           [line: 209]


In [376]:
%bp 30D6

Breakpoints
    1) ASCII_CONVERT: x3088: x202B  LD R0, CONVERT_VAL                        [line: 50]
    2) ASCII_BASE: x308B: x2229  LD R1, NEG_ONE                            [line: 57]
    3) ASCII_LOOP_REP: x3093: x3424  ST R2, TEMPY                              [line: 73]
    4)            x30D2: x14A1  ADD R2, R2, #1                            [line: 164]
    5) MULT_PRODUCT: x30D6: xFF9C  ;; Invalid TRAP vector: x009C             [line: 170]
    6)            x30F5: x3403  ST R2, QUOTIENT                           [line: 209]


In [561]:
%cont

...breakpoint hit at x30AB
Computation SUSPENDED
Instructions: 530
Cycles: 3814 (0.001907 milliseconds)

Registers:
PC: x30AB
N: 1 Z: 0 P: 0 
R0: xFFF6 R1: x0001 R2: x3066 R3: x0000 
R4: x0000 R5: x3002 R6: x30BD R7: x30AB 


In [550]:
%pc x3000


Registers:
PC: x3000
N: 0 Z: 1 P: 0 
R0: x0000 R1: x0000 R2: x0000 R3: x0000 
R4: x0000 R5: x0000 R6: x0000 R7: x0000 


In [597]:
%step

Stepping...  => read, <= write, (Instructions/Cycles):
    PC <= x045B
(566/4121) LDR R0, R1, 0 (x045B*: x6040)
  Reading memory[x04b0] (x000a) =>
    R0 <= x000A
    NZP <= (0, 0, 1)

Registers:
PC: x045B
N: 0 Z: 0 P: 1 
R0: x000A R1: x04B0 R2: xFFF6 R3: x0030 
R4: x0000 R5: x3002 R6: x30BD R7: x0490 


In [358]:
.ORIG x3000

ADD R0, R0, xC
ADD R1, R1, xA
LEA R6, Divide
LD R2, PUSH_IND1
JSRR R2         ; R0 / R1
LD R2, QUOTIENT

HALT

Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET
        
PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0        
        
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [359]:
%exe

Computation completed
Instructions: 67
Cycles: 497 (0.000249 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x000C R1: x000A R2: x0001 R3: x0000 
R4: x0000 R5: x3008 R6: x30AB R7: x3007 


In [442]:
%dis x3064

Memory disassembled:
           x3064: x0000 - \0
           x3065: x0000 - \0
PUSH:      x3066: x2B9A  LD R5, TOP                                [line: 9]
           x3067: x7140  STR R0, R5, 0                             [line: 10]
           x3068: x1B61  ADD R5, R5, #1                            [line: 11]
           x3069: x7340  STR R1, R5, 0                             [line: 12]
           x306A: x1B61  ADD R5, R5, #1                            [line: 13]
           x306B: x7540  STR R2, R5, 0                             [line: 14]
           x306C: x1B61  ADD R5, R5, #1                            [line: 15]
           x306D: x7740  STR R3, R5, 0                             [line: 16]
           x306E: x1B61  ADD R5, R5, #1                            [line: 17]
           x306F: x7940  STR R4, R5, 0                             [line: 18]
           x3070: x1B61  ADD R5, R5, #1                            [line: 19]
           x3071: x7D40  STR R6, R5, 0                          